# The Tensor
#### Previously, we made a distinction between vectors (one-dimensional arrays) and matrices (two-dimensional arrays). When we start working with more complicated neural networks, we'll need to use a higher-dimensional arrays as well.

#### In many neural network libraries, *n*-dimensional arrays are referred to as *tensors*, which is what we'll call them too.
#### If I were writing an entire book about deep learning, I'd implement a full-featured `Tensor` class that overloaded Python's arithmetic operators and could handle a variety of other operations. Such an implementation would take a notebook on its own. Here we'll cheat and say that a `Tensor` is just a `list`. This is true in one direction - all our vectors and matrices and higher-dimensional analogues *are* lists. It is certainly not true in the other direction - most Python *lists* are not *n*-dimensional arrays in our sense